In [2]:
from dotenv import load_dotenv
load_dotenv()

import os 
import os.path as osp
import json
import tiktoken
import ollama
from openai import AzureOpenAI
from transformers import AutoTokenizer
import pprint

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [3]:
azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = "gpt-4"

project_path = '/home/mou/Projects/COLING-LogicLLM/LogLM'
data_path = osp.join(project_path, 'data')
demonstration_path = osp.join(project_path, 'src/logicllm/prompts/baseline')
datasets = ['FOLIO', 'AR-LSAT', 'LogicalDeduction', 'ProntoQA', 'ProofWriter']

# Prompt example

In [5]:
dataset_name = 'FOLIO'
split = 'dev'
mode = 'CoT'
stop_words = "------"
label_phrase = 'The correct option is:'

def prompt_creator(in_context_example, test_example):
    full_prompt = in_context_example
    context = test_example['context'].strip()
    question = test_example['question'].strip()
    options = '\n'.join([opt.strip() for opt in test_example['options']])
    full_prompt = full_prompt.replace('[[CONTEXT]]', context)
    full_prompt = full_prompt.replace('[[QUESTION]]', question)
    full_prompt = full_prompt.replace('[[OPTIONS]]', options)
    return full_prompt

with open(os.path.join(data_path, dataset_name, f'{split}.json')) as f:
    raw_dataset = json.load(f)

with open(os.path.join(demonstration_path, f'{dataset_name}_{mode}.txt')) as f:
    in_context_examples = f.read()

print(in_context_examples)
    
example = raw_dataset[0]
question, answer = example['question'], example['answer']
full_prompt = prompt_creator(in_context_examples, example)

# print(full_prompt)

Given a problem statement as contexts, the task is to answer a logical reasoning question. 
------
Context:
The Blake McFall Company Building is a commercial warehouse listed on the National Register of Historic Places. The Blake McFall Company Building was added to the National Register of Historic Places in 1990. The Emmet Building is a five-story building in Portland, Oregon. The Emmet Building was built in 1915. The Emmet Building is another name for the Blake McFall Company Building. John works at the Emmet Building.

Question: Based on the above information, is the following statement true, false, or uncertain? The Blake McFall Company Building is located in Portland, Oregon.

Options:
A) True
B) False
C) Uncertain

Reasoning:
The Blake McFall Company Building is another name for the Emmet Building. The Emmet Building is located in Portland, Oregon. Therefore, the Blake McFall Company Building is located in Portland, Oregon.

The correct option is: A
------
Context:
People eat me

# OpenAI

In [12]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens
print(f'Number of tokens in prompt GPT-4: {num_tokens_from_string(full_prompt, "gpt-4")}')


client = AzureOpenAI(
    api_key=azure_openai_api_key,  
    api_version="2024-02-01",
    azure_endpoint = azure_openai_endpoint
)
response = client.chat.completions.create(
    model = deployment_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": full_prompt}
    ],
    temperature = 0.0,
    top_p = 1.0,
    stop = stop_words
)

generated_content = response.choices[0].message.content.strip()
print(generated_content)
generated_answer = generated_content.split(label_phrase)[-1].strip()
generated_reasoning = generated_content.split(label_phrase)[0].strip()


output_json = {'id': example['id'], 
            'question': question, 
            'answer': answer, 
            'predicted_reasoning': generated_reasoning,
            'predicted_answer': generated_answer}
print(f'id : {example["id"]}')
print(f'question : {question}')
print(f'answer : {answer}')
# print(f'predicted reasoning : {generated_reasoning}')
print(f'predicted answer : {generated_answer}')


Number of tokens in prompt GPT-4: 700
Bonnie either both attends and is very engaged with school events and is a student who attends the school, or she neither attends and is very engaged with school events nor is a student who attends the school. If people perform in school talent shows often, then they attend and are very engaged with school events. Therefore, if Bonnie performs in school talent shows often, then she attends and is very engaged with school events. Since Bonnie is a student who attends the school, it can be inferred that she performs in school talent shows often.

The correct option is: A
id : FOLIO_dev_0
question : Based on the above information, is the following statement true, false, or uncertain? Bonnie performs in school talent shows often.
answer : C
predicted answer : A


In [16]:
pprint.pprint(response.choices[0].message.content.strip())

('Bonnie either both attends and is very engaged with school events and is a '
 'student who attends the school, or she neither attends and is very engaged '
 'with school events nor is a student who attends the school. If people '
 'perform in school talent shows often, then they attend and are very engaged '
 'with school events. Therefore, if Bonnie performs in school talent shows '
 'often, then she attends and is very engaged with school events. Since Bonnie '
 'is a student who attends the school, it can be inferred that she performs in '
 'school talent shows often.\n'
 '\n'
 'The correct option is: A')


# Llama 3.1

In [17]:
llama31_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B")
tokens = llama31_tokenizer.encode(full_prompt)
print(f'Number of tokens in prompt Llama 3.1-8B: {len(tokens)}')

# stream = ollama.chat(
#     model='llama3.1',
#     messages=[
#         {"role": "system", "content": "You are a helpful assistant."},
#         {"role": "user", "content": full_prompt}
#     ],
#     stream=True,
# )
# for chunk in stream:
#   print(chunk['message']['content'], end='', flush=True)
stream = ollama.chat(
    model='llama3.1',
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": full_prompt}
    ],
    stream=False,
    options={
        'temperature': 0.0,
        # 'seed': 47,
    }
)
print(stream['message']['content'])

Number of tokens in prompt Llama 3.1-8B: 701
Let's analyze the context and question step by step:

1. If people perform in school talent shows often, then they attend and are very engaged with school events.
2. People either perform in school talent shows often or are inactive and disinterested members of their community.
3. All people who are inactive and disinterested members of their community chaperone high school dances.
4. Bonnie either both attends and is very engaged with school events and is a student who attends the school, or she neither attends and is very engaged with school events nor is a student who attends the school.

From statement 2, we know that Bonnie either performs in school talent shows often or is an inactive and disinterested member of her community. 

If Bonnie is an inactive and disinterested member of her community (which means she chaperones high school dances), then it would be incorrect to say that she performs in school talent shows often.

However, th